**Table of contents**<a id='toc0_'></a>    
- [ロボットアーム](#toc1_)    
- [pybulletの起動](#toc2_)    
- [pybulletの初期設定](#toc3_)    
- [ロボットアームの生成](#toc4_)    
- [関数の定義](#toc5_)    
  - [同次変換行列の定義](#toc5_1_)    
  - [ヤコビ行列の定義](#toc5_2_)    
  - [疑似逆行列の定義](#toc5_3_)    
- [シミュレーションの実行](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[ロボットアーム](#toc0_)

本notebookでは2軸のロボットアームを生成し、ヤコビ行列を用いた数値解法による逆運動学を求める方法を解説します。

![](../images/RobotArm/robot_arm_inverse_kinematics_numerical_method_%20using_jacobian_matrix/inverse_kinematics_jacobian_overview.png)

（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = Mesa
ven = Mesa


# <a id='toc3_'></a>[pybulletの初期設定](#toc0_)

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step) # 1stepあたりに経過する時間の設定

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# GUIモードの際のカメラの位置などを設定
camera_distance = 2.0
camera_yaw = 0.0 # deg
camera_pitch = -20 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# <a id='toc4_'></a>[ロボットアームの生成](#toc0_)
今回は、2軸ロボットアーム`simple_2d_arm.urdf`を生成します。  
ロボットは下図のような構成になっています（センサーについては、`robot_arm_sensor.ipynb`にて解説しています。）  

![](../images/RobotArm/2d_robot_arm.png)

In [3]:
# ロボットの読み込み
arm_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
arm_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
arm_id = pybullet.loadURDF("../urdf/simple2d_arm.urdf",arm_start_pos, arm_start_orientation, useFixedBase=True) # ロボットが倒れないように、useFixedBase=Trueでルートのリンクを固定

# GUIモードの際のカメラの位置などを設定
camera_distance = 1.5
camera_yaw = 180.0 # deg
camera_pitch = -10 # deg
camera_target_position = [0.0, 0.0, 1.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: target_position_vertual_link


# <a id='toc5_'></a>[関数の定義](#toc0_)

## <a id='toc5_1_'></a>[同次変換行列の定義](#toc0_)
同次変換行列を定義する関数を定義します

（詳細は、[robot_arm_homogeneous_matrix_forward_kinematics](https://github.com/akinami3/PybulletRobotics/blob/main/RobotArm/robot_arm_homogeneous_matrix_forward_kinematics_jp.ipynb)にて解説しています。）

In [4]:
import numpy as np

def Hz(theta):
    """
    2次元平面における回転行列を求める（2次元平面における回転はz軸周り）

    Parameters
    ----------
    theta : float
        回転角度[rad]

    Returns
    -------
    numpy.ndarray
        回転行列
    """
    return np.array([[np.cos(theta), -np.sin(theta), 0], 
                     [np.sin(theta), np.cos(theta), 0], 
                     [0, 0, 1]])

def Hp(x, y):
    """
    2次元平面における平行移動行列を求める

    Parameters
    ----------
    x : float
        x方向の平行移動量
    y : float
        y方向の平行移動量

    Returns
    -------
    numpy.ndarray
        平行移動行列
    """
    return np.array([[1, 0, x], 
                     [0, 1, y], 
                     [0, 0, 1]])

## <a id='toc5_2_'></a>[ヤコビ行列の定義](#toc0_)
[robot_arm_inverse_kinematics_analytical_method.ipynb](../RobotArm/robot_arm_inverse_kinematics_analytical_method_jp.ipynb)では、解析的に逆運動学を求めました。

解析的解法による逆運動学は、瞬時に解を求めることができますが、ロボットアームの構成が複雑な場合、解析的に解くことが難しい場合があります。

<br>

一方で、数値的解法による逆運動学では、「目標手先位置$\vec{P_{\mathrm{goal}}}$」方向に近づくように、「各関節角度$\vec{Q}$」を少しずつ動かしていくことで、解を求めるため、複雑な構成のロボットアームでも解を求めることができます。

![](../images/RobotArm/robot_arm_jacobian_inverse_kinemarics/inverse_kinematics_jacobian_overview.png)


<br>

この際、「手先位置の微小変化量$\Delta \vec{P}$」と「関節角度の微小変化量$\Delta \vec{Q}$」を関連付ける必要があるわけですが、ここで登場するのが「ヤコビ行列$J$」です。
$$
\Delta \vec{P} = J \Delta \vec{Q}
$$
![](../images/RobotArm/robot_arm_jacobian_inverse_kinemarics/delta_q_delta_p_jacpbian.png)

<br>

ヤコビ行列は、順運動学の数式を各関節角度で偏微分したものとして定義されます。
$$
J = \begin{bmatrix}
\frac{\partial x}{\partial \theta_1} & \frac{\partial x}{\partial \theta_2} & \cdots & \frac{\partial x}{\partial \theta_n} \\
\frac{\partial y}{\partial \theta_1} & \frac{\partial y}{\partial \theta_2} & \cdots & \frac{\partial y}{\partial \theta_n} \\
\frac{\partial z}{\partial \theta_1} & \frac{\partial z}{\partial \theta_2} & \cdots & \frac{\partial z}{\partial \theta_n} \\
\end{bmatrix}
$$

<br>

今回の2軸ロボットアームの場合、順運動学の数式は
$$
x = l_1 \cos(\theta_1) + l_2 \cos(\theta_1 + \theta_2) \\
y = l_1 \sin(\theta_1) + l_2 \sin(\theta_1 + \theta_2) \\
$$
となるため、ヤコビ行列は以下のようになります。
$$
J = \begin{bmatrix}
\frac{\partial x}{\partial \theta_1} & \frac{\partial x}{\partial \theta_2} \\
\frac{\partial y}{\partial \theta_1} & \frac{\partial y}{\partial \theta_2} \\
\end{bmatrix}
= \begin{bmatrix}
-l_1 \sin(\theta_1) - l_2 \sin(\theta_1 + \theta_2) & -l_2 \sin(\theta_1 + \theta_2) \\
l_1 \cos(\theta_1) + l_2 \cos(\theta_1 + \theta_2) & l_2 \cos(\theta_1 + \theta_2) \\
\end{bmatrix}
$$

---

- $l_1$：リンク1の長さ
- $l_2$：リンク2の長さ
- $\theta_1$：関節1の角度
- $\theta_2$：関節2の角度

---


<br>



これで、$J$が求まったため、
$$\Delta \vec{P} = J \Delta \vec{Q} （再掲）$$
を解くためには、$\Delta \vec{P}$と$\Delta \vec{Q}$のどちらかを求める必要がありますが、  
$\Delta \vec{P}$は下図のように、「目標手先位置$\vec{P_{\mathrm{goal}}}$」と「現在手先位置$\vec{P_{\mathrm{current}}}$」の差分として簡単に求めることができます。
（$P_{\mathrm{param}}$は、手先位置をどの程度微小に変化させるかを決めるパラメータです。）
$$
\Delta \vec{P} = P_{\mathrm{param}}(\vec{P_{\mathrm{goal}}} - \vec{P_{\mathrm{current}}})
$$
![](../images/RobotArm/robot_arm_jacobian_inverse_kinemarics/delta_p.png)
※ ここでは、$P_{\mathrm{param}}$はスカラー値として扱っていますが、$P_{\mathrm{param}} = \begin{bmatrix} P_{\mathrm{param_x}} & 0 \\ 0 & P_{\mathrm{param_y}} \end{bmatrix}$として、x,y成分に異なる値を設定することもあります。

<br>

ここまでで、「手先位置の微小変化量$\Delta \vec{P}$」と「ヤコビ行列$J$」が求まったため、最後に以下の式変換により「角関節角度の微小変化量$\Delta \vec{Q}$」を求めることができます。（ここで、$J^+$はヤコビ行列$J$の疑似逆行列を表します。）

$$
\Delta \vec{P} = J \Delta \vec{Q} \\
\Delta \vec{Q} = J^+ \Delta \vec{P}
$$

<br>

上記の式を用いて、「現在手先位置$\vec{P_{\mathrm{current}}}$」が「目標手先位置$\vec{P_{\mathrm{goal}}}$」に収束するまで、関節角度$\vec{Q}$を少しずつ動かしていくことで、解を求めることができます。


In [5]:
import math
def make_jacobian_matrix(link1_length, theta1, link2_length, theta2):
    """
    2次元平面における2リンクアームのヤコビ行列を求める
    J = [[∂x/∂θ1, ∂x/∂θ2],
         [∂y/∂θ1, ∂y/∂θ2]]

    Parameters
    ----------
    link1_length : float
        リンク1の長さ
    link2_length : float
        リンク2の長さ
    theta1 : float
        リンク1の回転角度(rad)
    theta2 : float
        リンク2の回転角度(rad)

    Returns
    -------
    J : numpy.ndarray
        2次元平面における2リンクアームのヤコビ行列
    """

    J = np.array([[-link1_length*math.sin(theta1) - link2_length*math.sin(theta1+theta2), -link2_length*math.sin(theta1+theta2)],
                  [ link1_length*math.cos(theta1) + link2_length*math.cos(theta1+theta2),  link2_length*math.cos(theta1+theta2)]])
    return J


## <a id='toc5_3_'></a>[疑似逆行列の定義](#toc0_)

上記で述べたように、$\Delta \vec{Q}$を求める際に、ヤコビ行列$J$の疑似逆行列$J^+$が必要となるため、ここで疑似逆行列を求める関数を定義します。

In [6]:
def make_inverse_matrix(mat):
    """
    2×2行列の逆行列を求める（疑似逆行列を求める際に使用する）
    （numpyのlinalg.inv関数を使っても良いですが、ここではアルゴリズムを示すために自作で実装）

    Parameters
    ----------
    mat: numpy.ndarray
         2×2行列

    Returns
    -------
    inverse_mat : numpy.ndarray
                  2×2行列の逆行列
    """
    inverse_mat = 1/(mat[0,0]*mat[1,1] - mat[0,1]*mat[1,0]) * np.array([[mat[1,1], -mat[0,1]],
                                                                        [-mat[1,0], mat[0,0]]])
    return inverse_mat

def make_trans_matrix(mat):
    """
    2×2行列の転置行列を求める（疑似逆行列を求める際に使用する）
    （numpyのtranspose関数を使っても良いが、ここではアルゴリズムを示すために自作で実装）

    Parameters
    ----------
    mat: numpy.ndarray
         2×2行列

    Returns
    -------
    trans_mat : numpy.ndarray
                2次元平面における2リンクアームのヤコビ行列
    """
    mat_minus =  np.array([[mat[0,0], mat[1,0]],
                           [mat[0,1], mat[1,1]]])
    return mat_minus

def make_pseudo_inverse_matrix(mat):
    """
    疑似逆行列を求める
    （numpyのlinalg.pinv関数を使っても良いが、ここではアルゴリズムを示すために自作で実装）

    Parameters
    ----------
    mat: numpy.ndarray
         行列

    Returns
    -------
    inverse_mat : numpy.ndarray
                  疑似逆行列
    """
    trans_mat = make_trans_matrix(mat)
    pseudo_inverse_mat = make_inverse_matrix(trans_mat @ mat) @ trans_mat
    return pseudo_inverse_mat

# <a id='toc6_'></a>[シミュレーションの実行](#toc0_)

ここまでの解説をまとめると、ヤコビ行列を用いた数値解法による逆運動学の手順は以下の通りです。
1. 「目標手先位置$\vec{P_{\mathrm{goal}}}$」を設定する
2. 「現在手先位置$\vec{P_{\mathrm{current}}}$」から「目標手先位置$\vec{P_{\mathrm{goal}}}$」へ向かう「手先位置の微小変化量$\Delta \vec{P}$」を求める
3. 「ヤコビ行列$J$」を求める
4. 「ヤコビ行列$J$」の疑似逆行列$J^+$を用いて、「関節角度の微小変化量$\Delta \vec{Q}$」を求める
5. 「現在の各関節角度$\vec{Q_{\mathrm{current}}}$」を「関節角度の微小変化量$\Delta \vec{Q}$」だけ動かし「更新後の各関節角度$\vec{Q_{\mathrm{new}}}$」とする
6. 「更新後の各関節角度$\vec{Q_{\mathrm{new}}}$」に基づいて、順運動学により「更新後の手先位置$\vec{P_{\mathrm{new}}}$」を求める
7. 「現在手先位置$\vec{P_{\mathrm{current}}}$」=「更新後の手先位置$\vec{P_{\mathrm{new}}}$」、「現在の各関節角度$\vec{Q_{\mathrm{current}}}$」=「更新後の各関節角度$\vec{Q_{\mathrm{new}}}$」と更新する
8. 「現在手先位置$\vec{P_{\mathrm{current}}}$」が「目標手先位置$\vec{P_{\mathrm{goal}}}$」に収束するまで、2~7の手順を繰り返す

下記コードを実行すると、上記の手順で逆運動学が実行されます。


---

※ 今回は分かりやすさのため、ロボットアームを少しずつ動かす様子をpybullet上で可視化していますが、以下コードの92-94行目をコメントアウトして、可視化部分を省略することで、逆運動学の計算のみを行うことも可能です。   

【コード該当箇所】
```python
pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, Q_current[0][0]) # 関節角度を更新
pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, Q_current[1][0]) # 関節角度を更新
time.sleep(time_step)
```

---


In [7]:
import time
import sys

# ロボットのリンクの長さ
LINK1_LENGTH = 0.5   # link1の長さ(「simple2d_arm.urdf」のlink1のz方向の長さ)
LINK2_LENGTH = 0.55  # link2の長さ(「simple2d_arm.urdf」のlink2+force_sensor_linkのz方向の長さ)

# ロボットの各関節のインデックス
LINK1_JOINT_IDX = 0
LINK2_JOINT_IDX = 1

# 初期設定（この箇所を変更すると、結果が変わります） ========================
# 最大繰り返し回数
max_loop_num = 1000

# 各リンクの初期角度（特異姿勢にならないように注意）
theta1_deg = 0.0
theta2_deg = -1.0

# 【手順1】目標手先位置（実現できない位置を指定すると解が求まらないので注意）
xe_goal = 0.0
ye_goal = 0.8
P_goal = np.array([[xe_goal],
                    [ye_goal]]) # （初期）目標手先位置P_goal（ここは変更しない）

# 手先位置Pを各ループごとにどの程度動かすかを決める定数（値が大きいほど早く収束しようとするが安定しない、小さいほど収束がゆっくりだが安定する）
P_delta_param = 0.005

# 目標手先位置に到達したと判定する閾値(P_currentとP_goalの距離がどの程度まで近づいたらゴールとみなすか)
goal_dis = 0.01
# ==========================================================================

# 関節角度をラジアンに変換
link1_angle_rad  = np.deg2rad(theta1_deg) 
link2_angle_rad  = np.deg2rad(theta2_deg)

# 関節角度を初期設定
Q_current = np.array([[link1_angle_rad],
                      [link2_angle_rad]]) # 関節角度Q_current

# 初期関節角度にロボットの関節角度を設定
pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, Q_current[0][0]) # 関節角度を更新
pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, Q_current[1][0]) # 関節角度を更新

# 同次変換行列による順運動学により、初期の手先位置P_currentを計算
H12 = Hz(Q_current[0][0]) @ Hp(LINK1_LENGTH, 0) # H12：link「1」座標系 -> link「2」座標系
H2e = Hz(Q_current[1][0]) @ Hp(LINK2_LENGTH, 0) # H2e：link「2」座標系 ->「e」nd effector座標系
H1e = H12@H2e # H1e：link「1」座標系 ->「e」nd effector座標系
x1, y1 = 0, 0 # link1座標系の原点
oe = H1e@np.array([x1, y1, 1])
xe, ye = oe[0], oe[1]
P_current = np.array([[xe],
                      [ye]]) # 現在の手先位置P_current

# ヤコビ行列を用いた数値解法による逆運動学を実行
for i in range(max_loop_num):
    # 現在の手先P_current→目標手先P_goal 方向のベクトル
    P_current_to_P_goal = P_goal - P_current

    # 【手順2】：現在の手先P_current→目標手先P_goal 方向に向かう微小量ΔP
    P_delta = P_current_to_P_goal * P_delta_param

    # 【手順3】：ヤコビ行列を計算
    jacobian = make_jacobian_matrix(LINK1_LENGTH, Q_current[0][0], LINK2_LENGTH, Q_current[1][0]) # ヤコビ行列J
    
    # ヤコビ行列の擬似逆行列を計算
    jacobian_inverse = make_pseudo_inverse_matrix(jacobian) 

    # 【手順4】：ΔP移動するために必要な「各関節の、微小角度ΔQ」を計算
    Q_delta = jacobian_inverse @ P_delta

    # 【手順5】：ΔQだけ各関節を動かす
    Q_new = Q_current + Q_delta
    Q_new = Q_new % (2*math.pi) #（関節角度が0～2πの間に収まるようにするための処理）

    # 【手順6】：更新後の関節角度Q_newを用いて、手先位置P_newを計算（順運動学）
    H12 = Hz(Q_new[0][0]) @ Hp(LINK1_LENGTH, 0) # T12：link「1」座標系 -> link「2」座標系
    H2e = Hz(Q_new[1][0]) @ Hp(LINK2_LENGTH, 0) # T2e：link「2」座標系 ->「e」nd effector座標系
    H1e = H12@H2e # T1e：link「1」座標系 ->「e」nd effector座標系
    x1, y1 = 0, 0 # link1座標系の原点
    oe = H1e@np.array([x1, y1, 1])
    xe, ye = oe[0], oe[1]
    P_new = np.array([[xe],
                      [ye]]) # 更新後の手先位置P_new
    sys.stdout.write("\r目標手先位置 P_goal：({:.2f}, {:.2f})、現在手先位置 P_current：({:.2f}, {:.2f})       ".format(P_goal[0][0], P_goal[1][0], P_current[0][0], P_current[1][0]))

    # 【手順7】：現在関節角度Q_current、現在手先位置P_currentを更新
    Q_current = Q_new
    P_current = P_new

    # 現在関節角度Q_currentに、ロボットの関節角度を設定
    pybullet.resetJointState(arm_id, LINK1_JOINT_IDX, Q_current[0][0]) # 関節角度を更新
    pybullet.resetJointState(arm_id, LINK2_JOINT_IDX, Q_current[1][0]) # 関節角度を更新
    time.sleep(time_step)

    # 【手順8】：目標手先P_goalに到達したら終了
    if (P_goal[0][0]-P_current[0][0])**2  + (P_goal[1][0]-P_current[1][0])**2 <  goal_dis**2: # 2点間の距離の公式を使用（ただし、sqrt関数は処理が重いので、両辺を2乗した値で比較）
        break

# 繰り返し回数 が ループの最大回数 の場合、解が見つからなかったとして終了
if i == max_loop_num-1:
    print("\r位置を計算できませんでした（特異点，もしくは実現不可能な座標の可能性があります）                ")
else:
    print("\r【最終結果】                                                                                   ")
    print("ループ回数：{}回目, 標手先位置 P_goal：({:.2f}, {:.2f})、現在手先位置 P_current：({:.2f}, {:.2f})".format(i+1, P_goal[0][0], P_goal[1][0], P_current[0][0], P_current[1][0]))

【最終結果】                                                                                   
ループ回数：955回目, 標手先位置 P_goal：(0.00, 0.80)、現在手先位置 P_current：(0.01, 0.79)
